In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from itertools import product
from tqdm import tqdm
# Попытка импортировать Prophet
try:
    from prophet import Prophet
except ImportError:
    print("Библиотека Prophet не установлена. Для ее использования выполните: pip install prophet")
    Prophet = None

Библиотека Prophet не установлена. Для ее использования выполните: pip install prophet


In [11]:
!pip install Prophet

   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
    --------------------------------------- 0.3/13.3 MB ? eta -:--:--
    --------------------------------------- 0.3/13.3 MB ? eta -:--:--
    --------------------------------------- 0.3/13.3 MB ? eta -:--:--
    --------------------------------------- 0.3/13.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.3 MB 284.3 kB/s eta 0:00:46
   - -------------------------------------- 0.5/13.3 MB 284.3 kB/s eta 0:00:46
   -- ------------------------------------- 0.8/13.3 MB 394.8 kB/s eta 0:00:32
   -- ------------------------------------- 0.8/13.3 MB 394.8 kB/s eta 0:00:32
   --- ------------------------------------ 1.0/13.3 MB 470.4 kB/s eta 0:00:27
   --- ------------------------------------ 1

In [7]:
class TimeSeriesPredictor:
    def __init__(self, filepath):
        self.filepath = filepath
        self.df = None
        self.time_column = None
        self.value_column = None
        self.is_seasonal = False
        self.seasonal_period = None
#Настя
    def load_data(self, time_column_name=None, value_column_name=None):
        """
        Загружает данные из CSV файла и подготавливает их.
        Предполагается, что данные имеют временной столбец и столбец значений.
        """
        try:
            self.df = pd.read_csv(self.filepath)
            print(f"Данные успешно загружены из: {self.filepath}")
            print("Первые 5 строк данных:")
            print(self.df.head())

            # Попытка автоматического определения столбцов
            if not time_column_name:
                # Ищем столбец, который можно преобразовать в datetime
                for col in self.df.columns:
                    try:
                        self.df[col] = pd.to_datetime(self.df[col])
                        self.time_column = col
                        break
                    except (ValueError, TypeError):
                        pass
                if not self.time_column:
                    raise ValueError("Не удалось автоматически определить временной столбец. Укажите его явно.")
            else:
                self.df[time_column_name] = pd.to_datetime(self.df[time_column_name])
                self.time_column = time_column_name

            if not value_column_name:
                # Ищем числовой столбец, который не является временным
                numeric_cols = self.df.select_dtypes(include=np.number).columns.tolist()
                numeric_cols = [col for col in numeric_cols if col != self.time_column]
                if not numeric_cols:
                    raise ValueError("Не удалось автоматически определить столбец значений. Укажите его явно.")
                elif len(numeric_cols) > 1:
                    print(f"Обнаружено несколько числовых столбцов: {numeric_cols}. Использую первый: {numeric_cols[0]}.")
                    self.value_column = numeric_cols[0]
                else:
                    self.value_column = numeric_cols[0]
            else:
                self.value_column = value_column_name

            self.df = self.df[[self.time_column, self.value_column]].set_index(self.time_column).sort_index()
            print(f"Используемый временной столбец: '{self.time_column}'")
            print(f"Используемый столбец значений: '{self.value_column}'")
            print("\nИнформация о DataFrame после обработки:")
            self.df.info()
            print("\nСтатистика по столбцу значений:")
            print(self.df[self.value_column].describe())

        except Exception as e:
            print(f"Ошибка при загрузке или обработке данных: {e}")
            self.df = None

    def plot_time_series(self, title="Временной ряд"):
        """Визуализирует временной ряд."""
        if self.df is None:
            print("Данные не загружены.")
            return

        plt.figure(figsize=(15, 6))
        plt.plot(self.df.index, self.df[self.value_column], label=self.value_column)
        plt.title(title)
        plt.xlabel("Дата")
        plt.ylabel("Значение")
        plt.legend()
        plt.grid(True)
        plt.show()
 # Женя
    def check_for_seasonality_and_trend(self):
        """
        Проверяет наличие сезонности и тренда, используя разложение временного ряда и тест Дики-Фуллера.
        """
        if self.df is None:
            print("Данные не загружены.")
            return

        print("\n--- Проверка на сезонность и тренд ---")

        # Проверка на пропуски
        if self.df[self.value_column].isnull().any():
            print("В данных обнаружены пропуски. Рекомендуется их обработать (например, интерполяцией).")
            # Пример простой интерполяции:
            self.df[self.value_column] = self.df[self.value_column].interpolate(method='time')
            print("Пропуски интерполированы.")

        # Тест Дики-Фуллера для проверки стационарности (наличия тренда)
        try:
            result = adfuller(self.df[self.value_column].dropna())
            print(f"ADF Statistic: {result[0]}")
            print(f"p-value: {result[1]}")
            print("Critical Values:")
            for key, value in result[4].items():
                print(f"\t{key}: {value}")

            if result[1] > 0.05:
                print("Временной ряд нестационарен (вероятно, присутствует тренд).")
            else:
                print("Временной ряд стационарен (тренд не выражен).")
        except Exception as e:
            print(f"Ошибка при выполнении теста Дики-Фуллера: {e}")

        # Разложение временного ряда
        # Попытка определить частоту (period)
        freq_str = pd.infer_freq(self.df.index)
        print(f"Предполагаемая частота данных: {freq_str}")

        # Попытка определить сезонный период
        possible_periods = []
        if freq_str and 'D' in freq_str: # Если дневные данные
            possible_periods = [7, 30, 365]
        elif freq_str and 'M' in freq_str: # Если месячные данные
            possible_periods = [12]
        elif freq_str and 'H' in freq_str: # Если часовые данные
            possible_periods = [24, 24*7] # Ежедневная, еженедельная

        print(f"Попытка разложения с периодами: {possible_periods}")
        decomposition_successful = False
        for period in possible_periods:
            if len(self.df) > 2 * period: # Должно быть достаточно данных для разложения
                try:
                    # Попытка аддитивной модели
                    decomposition = seasonal_decompose(self.df[self.value_column].dropna(), model='additive', period=period)
                    decomposition.plot().set_size_inches(10, 8)
                    plt.suptitle(f"Разложение временного ряда (Аддитивная модель, период={period})", y=1.02)
                    plt.tight_layout(rect=[0, 0, 1, 0.98])
                    plt.show()

                    # Попытка мультипликативной модели (если значения положительны)
                    if (self.df[self.value_column] > 0).all():
                        decomposition = seasonal_decompose(self.df[self.value_column].dropna(), model='multiplicative', period=period)
                        decomposition.plot().set_size_inches(10, 8)
                        plt.suptitle(f"Разложение временного ряда (Мультипликативная модель, период={period})", y=1.02)
                        plt.tight_layout(rect=[0, 0, 1, 0.98])
                        plt.show()
                    decomposition_successful = True
                    self.is_seasonal = True
                    self.seasonal_period = period
                    print(f"Сезонность предположительно присутствует с периодом: {period}")
                    break # Останавливаемся на первом успешном разложении
                except Exception as e:
                    pass # Пробуем следующий период

        if not decomposition_successful:
            print("Не удалось однозначно определить сезонность с помощью разложения. Возможно, сезонность отсутствует или данные недостаточны.")
            self.is_seasonal = False

        # ACF и PACF
        if result[1] > 0.05: # Если нестационарный
            diff_series = self.df[self.value_column].diff().dropna()
            print("\nACF и PACF для дифференцированного ряда (для определения порядков ARIMA):")
        else:
            diff_series = self.df[self.value_column].dropna()
            print("\nACF и PACF для исходного ряда (для определения порядков ARIMA):")

        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        plot_acf(diff_series, ax=axes[0], lags=min(40, len(diff_series)//2 -1))
        axes[0].set_title('Autocorrelation Function (ACF)')
        plot_pacf(diff_series, ax=axes[1], lags=min(40, len(diff_series)//2 -1))
        axes[1].set_title('Partial Autocorrelation Function (PACF)')
        plt.show()
        print("Посмотрите на графики ACF и PACF для определения потенциальных порядков p, d, q (для ARIMA) и P, D, Q, S (для SARIMA).")
        print("Значимые всплески на ACF указывают на порядок MA (q), на PACF - на порядок AR (p).")
        print("Периодические всплески на ACF/PACF могут указывать на сезонность.")
#Настя
    def _find_best_arima_params(self, train_data, max_p=3, max_d=2, max_q=3):
        """Автоматический подбор параметров ARIMA с использованием поиска по сетке."""
        best_aic = np.inf
        best_order = None
        best_model = None

        # Генерируем все возможные комбинации p, d, q
        p_range = range(0, max_p + 1)
        d_range = range(0, max_d + 1)
        q_range = range(0, max_q + 1)

        total_combinations = len(p_range) * len(d_range) * len(q_range)
        print(f"\n--- Поиск лучших параметров ARIMA (всего {total_combinations} комбинаций) ---")

        with tqdm(total=total_combinations, desc="Подбор параметров ARIMA") as pbar:
            for p, d, q in product(p_range, d_range, q_range):
                try:
                    model = ARIMA(train_data, order=(p, d, q))
                    model_fit = model.fit()
                    current_aic = model_fit.aic

                    if current_aic < best_aic:
                        best_aic = current_aic
                        best_order = (p, d, q)
                        best_model = model_fit
                except:
                    continue
                finally:
                    pbar.update(1)

        print(f"\nЛучшие параметры ARIMA: order={best_order} с AIC={best_aic:.2f}")
        return best_order, best_model

    def _find_best_sarima_params(self, train_data, seasonal_period, max_p=2, max_d=1, max_q=2, max_P=1, max_D=1, max_Q=1):
        """Автоматический подбор параметров SARIMA с использованием поиска по сетке."""
        best_aic = np.inf
        best_order = None
        best_seasonal_order = None
        best_model = None

        # Генерируем все возможные комбинации p, d, q и P, D, Q
        p_range = range(0, max_p + 1)
        d_range = range(0, max_d + 1)
        q_range = range(0, max_q + 1)
        P_range = range(0, max_P + 1)
        D_range = range(0, max_D + 1)
        Q_range = range(0, max_Q + 1)

        total_combinations = len(p_range) * len(d_range) * len(q_range) * len(P_range) * len(D_range) * len(Q_range)
        print(f"\n--- Поиск лучших параметров SARIMA (всего {total_combinations} комбинаций) ---")

        with tqdm(total=total_combinations, desc="Подбор параметров SARIMA") as pbar:
            for p, d, q, P, D, Q in product(p_range, d_range, q_range, P_range, D_range, Q_range):
                try:
                    model = ARIMA(train_data, order=(p, d, q), seasonal_order=(P, D, Q, seasonal_period))
                    model_fit = model.fit()
                    current_aic = model_fit.aic

                    if current_aic < best_aic:
                        best_aic = current_aic
                        best_order = (p, d, q)
                        best_seasonal_order = (P, D, Q, seasonal_period)
                        best_model = model_fit
                except:
                    continue
                finally:
                    pbar.update(1)

        print(f"\nЛучшие параметры SARIMA: order={best_order}, seasonal_order={best_seasonal_order} с AIC={best_aic:.2f}")
        return best_order, best_seasonal_order, best_model

    def _find_best_holtwinters_params(self, train_data, seasonal_period=None):
        """Автоматический подбор параметров модели Хольта-Уинтерса."""
        best_aic = np.inf
        best_trend = None
        best_seasonal = None
        best_model = None

        trend_options = ['add', 'mul', None]
        seasonal_options = ['add', 'mul', None] if seasonal_period else [None]

        total_combinations = len(trend_options) * len(seasonal_options)
        print(f"\n--- Поиск лучших параметров Хольта-Уинтерса (всего {total_combinations} комбинаций) ---")

        with tqdm(total=total_combinations, desc="Подбор параметров Хольта-Уинтерса") as pbar:
            for trend, seasonal in product(trend_options, seasonal_options):
                try:
                    if seasonal_period and seasonal:
                        model = ExponentialSmoothing(
                            train_data,
                            trend=trend,
                            seasonal=seasonal,
                            seasonal_periods=seasonal_period
                        )
                    else:
                        model = ExponentialSmoothing(
                            train_data,
                            trend=trend,
                            seasonal=None
                        )

                    model_fit = model.fit()
                    current_aic = model_fit.aic

                    if current_aic < best_aic:
                        best_aic = current_aic
                        best_trend = trend
                        best_seasonal = seasonal
                        best_model = model_fit
                except:
                    continue
                finally:
                    pbar.update(1)

        print(f"\nЛучшие параметры Хольта-Уинтерса: trend={best_trend}, seasonal={best_seasonal} с AIC={best_aic:.2f}")
        return best_trend, best_seasonal, best_model
#Женя
    def train_and_predict(self, forecast_horizon, model_type="auto",
                          order=None, seasonal_order=None,
                          holtwinter_trend=None, holtwinter_seasonal=None, holtwinter_seasonal_period=None,
                          auto_tune=True):
        """
        Обучает выбранную модель и делает прогноз.
        :param forecast_horizon: Длина горизонта прогнозирования.
        :param model_type: Тип модели ('auto', 'arima', 'sarima', 'holt_winters', 'prophet').
        :param order: Порядок (p,d,q) для ARIMA/SARIMA.
        :param seasonal_order: Сезонный порядок (P,D,Q,S) для SARIMA.
        :param holtwinter_trend: Тип тренда для Хольта-Уинтерса ('add', 'mul', 'add_damped', 'mul_damped', 'None').
        :param holtwinter_seasonal: Тип сезонности для Хольта-Уинтерса ('add', 'mul', 'None').
        :param holtwinter_seasonal_period: Период сезонности для Хольта-Уинтерса.
        :param auto_tune: Если True, автоматически подбирает лучшие параметры модели.
        :return: DataFrame с прогнозом.
        """
        if self.df is None:
            print("Данные не загружены.")
            return None

        print(f"\n--- Прогнозирование с использованием {model_type} ---")

        # Разделение на обучающую и тестовую выборки
        train_size = int(len(self.df) * 0.8)
        train_data = self.df[self.value_column].iloc[:train_size]
        test_data = self.df[self.value_column].iloc[train_size:]

        forecast_index = pd.date_range(start=self.df.index[-1], periods=forecast_horizon + 1, freq=pd.infer_freq(self.df.index))[1:]

        model = None
        forecast = None
        model_name = ""

        if model_type == "auto":
            if self.is_seasonal and self.seasonal_period:
                model_type = "sarima"
                print(f"Автоматически выбрана модель SARIMA из-за обнаруженной сезонности (период: {self.seasonal_period}).")
            else:
                model_type = "arima"
                print("Автоматически выбрана модель ARIMA (сезонность не обнаружена).")

        if model_type == "arima":
            try:
                model_name = "ARIMA"

                if auto_tune or order is None:
                    best_order, best_model = self._find_best_arima_params(train_data)
                    if best_order:
                        order = best_order
                        model_fit = best_model
                    else:
                        print("Не удалось подобрать параметры ARIMA. Использую параметры по умолчанию (1,1,1).")
                        order = (1,1,1)
                        model = ARIMA(train_data, order=order)
                        model_fit = model.fit()
                else:
                    model = ARIMA(train_data, order=order)
                    model_fit = model.fit()

                print(model_fit.summary())
                forecast = model_fit.predict(start=len(train_data), end=len(train_data) + forecast_horizon - 1)
                forecast.index = forecast_index
            except Exception as e:
                print(f"Ошибка при обучении модели ARIMA: {e}")
                return None
        elif model_type == "sarima":
            if not self.is_seasonal or not self.seasonal_period:
                print("Для SARIMA требуется сезонность. Пожалуйста, запустите `check_for_seasonality_and_trend` сначала.")
                if not self.seasonal_period:
                    self.seasonal_period = seasonal_order[3] if seasonal_order and len(seasonal_order) == 4 else 12
                print(f"Используется сезонный период: {self.seasonal_period}")
            try:
                model_name = "SARIMA"

                if auto_tune or order is None or seasonal_order is None:
                    best_order, best_seasonal_order, best_model = self._find_best_sarima_params(
                        train_data,
                        self.seasonal_period if self.seasonal_period else 12
                    )
                    if best_order and best_seasonal_order:
                        order = best_order
                        seasonal_order = best_seasonal_order
                        model_fit = best_model
                    else:
                        print("Не удалось подобрать параметры SARIMA. Использую параметры по умолчанию.")
                        order = (1,1,1)
                        seasonal_order = (1,1,1, self.seasonal_period if self.seasonal_period else 12)
                        model = ARIMA(train_data, order=order, seasonal_order=seasonal_order)
                        model_fit = model.fit()
                else:
                    seasonal_order_used = (
                        seasonal_order[0], seasonal_order[1], seasonal_order[2],
                        self.seasonal_period if self.seasonal_period else seasonal_order[3]
                    )
                    model = ARIMA(train_data, order=order, seasonal_order=seasonal_order_used)
                    model_fit = model.fit()

                print(f"Порядок SARIMA: {order}, Сезонный порядок: {seasonal_order}")
                print(model_fit.summary())
                forecast = model_fit.predict(start=len(train_data), end=len(train_data) + forecast_horizon - 1)
                forecast.index = forecast_index
            except Exception as e:
                print(f"Ошибка при обучении модели SARIMA: {e}")
                return None
 #Настя               
        elif model_type == "holt_winters":
            try:
                model_name = "Хольта-Уинтерса"

                if auto_tune or holtwinter_trend is None or holtwinter_seasonal is None:
                    best_trend, best_seasonal, best_model = self._find_best_holtwinters_params(
                        train_data,
                        self.seasonal_period if self.seasonal_period else holtwinter_seasonal_period
                    )
                    holtwinter_trend = best_trend
                    holtwinter_seasonal = best_seasonal
                    model_fit = best_model
                else:
                    if not holtwinter_seasonal_period:
                        holtwinter_seasonal_period = self.seasonal_period if self.seasonal_period else 12
                        print(f"Используется сезонный период Хольта-Уинтерса: {holtwinter_seasonal_period}")

                    model = ExponentialSmoothing(
                        train_data,
                        trend=holtwinter_trend,
                        seasonal=holtwinter_seasonal,
                        seasonal_periods=holtwinter_seasonal_period
                    )
                    model_fit = model.fit()

                print(model_fit.summary())
                forecast = model_fit.forecast(forecast_horizon)
                forecast.index = forecast_index
            except Exception as e:
                print(f"Ошибка при обучении модели Хольта-Уинтерса: {e}")
                return None
        elif model_type == "prophet":
            if Prophet is None:
                print("Prophet не установлен. Пожалуйста, установите его (pip install prophet) или выберите другую модель.")
                return None
            try:
                model_name = "Prophet"
                prophet_df = train_data.reset_index()
                prophet_df.columns = ['ds', 'y']

                model = Prophet()
                model.fit(prophet_df)

                future = model.make_future_dataframe(periods=forecast_horizon)
                prophet_forecast = model.predict(future)
                forecast = prophet_forecast['yhat'].iloc[-forecast_horizon:]
                forecast.index = forecast_index

                fig = model.plot(prophet_forecast)
                plt.title(f"Прогноз с помощью Prophet (Horizon: {forecast_horizon})")
                plt.xlabel("Дата")
                plt.ylabel("Значение")
                plt.show()

                fig2 = model.plot_components(prophet_forecast)
                plt.title("Компоненты Prophet (Тренд, Сезонность)")
                plt.show()

            except Exception as e:
                print(f"Ошибка при обучении модели Prophet: {e}")
                return None
        else:
            print(f"Неизвестный тип модели: {model_type}")
            return None

        if forecast is not None:
            forecast_df = pd.DataFrame({'Прогноз': forecast.values}, index=forecast.index)

            # Оценка качества на тестовых данных
            if len(test_data) > 0:
                actual_values = test_data.head(forecast_horizon)
                predicted_values = forecast.head(len(actual_values))

                if len(actual_values) > 0:
                    mae = mean_absolute_error(actual_values, predicted_values)
                    rmse = np.sqrt(mean_squared_error(actual_values, predicted_values))
                    print(f"\n--- Оценка качества для модели {model_name} ---")
                    print(f"MAE (Mean Absolute Error): {mae:.2f}")
                    print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")

            print(f"\nПрогноз на горизонт {forecast_horizon}:")
            print(forecast_df)

            # Визуализация
            plt.figure(figsize=(15, 7))
            plt.plot(self.df.index, self.df[self.value_column], label='Фактические данные')
            plt.plot(train_data.index, train_data, label='Обучающая выборка', color='blue', linestyle='--')
            if len(test_data) > 0:
                 plt.plot(test_data.index, test_data, label='Тестовая выборка', color='orange', linestyle='--')
            plt.plot(forecast_df.index, forecast_df['Прогноз'], label=f'Прогноз {model_name}', color='red', linestyle='--')
            plt.title(f"Фактические данные и прогноз ({model_name})")
            plt.xlabel("Дата")
            plt.ylabel("Значение")
            plt.legend()
            plt.grid(True)
            plt.show()

            return forecast_df
        return None

In [ ]:

# --- Пример использования ---
if __name__ == "__main__":
    # Читаем CSV и переименовываем колонки
    df = pd.read_csv('/content/daily-foreign-exchange-rates-31-.csv')

    # Переименовываем второй столбец с длинным названием
    df.columns = ['datesold', 'price']

    # Преобразуем колонку с датой в формат datetime
    df['datesold'] = pd.to_datetime(df['datesold'], errors='coerce')

    # Удаляем строки с отсутствующими значениями (если есть)
    df = df.dropna()

    # Сохраняем подготовленный CSV
    df.to_csv("sample_time_series.csv", index=False)

    # Инициализация предсказателя
    predictor = TimeSeriesPredictor("sample_time_series.csv")

    # Загрузка данных
    predictor.load_data()

    # Визуализация исходного ряда
    predictor.plot_time_series("Исходный временной ряд")

    # Проверка на сезонность и тренд
    predictor.check_for_seasonality_and_trend()

    # --- Выбор и запуск модели ---
    forecast_horizon = 365 # Прогноз на 365 точек вперед

    # 1. Прогноз с помощью SARIMA (если есть сезонность)
    if predictor.is_seasonal and predictor.seasonal_period:
        print(f"\n--- Пробуем SARIMA с периодом {predictor.seasonal_period} ---")
        sarima_forecast = predictor.train_and_predict(
            forecast_horizon=forecast_horizon,
            model_type="sarima",
            auto_tune=True # Автоматический подбор параметров
        )
    else:
        print("\n--- Сезонность не обнаружена, SARIMA не будет использоваться ---")

    # 2. Прогноз с помощью Хольта-Уинтерса
    print("\n--- Пробуем Модель Хольта-Уинтерса ---")
    holt_winters_forecast = predictor.train_and_predict(
        forecast_horizon=forecast_horizon,
        model_type="holt_winters",
        auto_tune=True # Автоматический подбор параметров
    )

    # 3. Прогноз с помощью Prophet (если установлен)
    if Prophet:
        print("\n--- Пробуем Prophet ---")
        prophet_forecast = predictor.train_and_predict(
            forecast_horizon=forecast_horizon,
            model_type="prophet"
        )
    else:
        print("\n--- Prophet не установлен, прогноз с Prophet пропущен ---")

    # 4. Прогноз с помощью ARIMA (как базовый вариант, если нет сильной сезонности)
    print("\n--- Пробуем ARIMA ---")
    arima_forecast = predictor.train_and_predict(
        forecast_horizon=forecast_horizon,
        model_type="arima",
        auto_tune=True # Автоматический подбор параметров
    )

    print("\nПрогнозирование завершено.")

# В целом, задания в работе были распределены 50 на 50